In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn import metrics
from sklearn.metrics import roc_auc_score

# import matplotlib.pyplot as plt

from sklearn.externals import joblib

In [2]:
import matplotlib.pyplot as plt

In [3]:
from sklearn.metrics import accuracy_score

In [4]:
import pandas as pd
import numpy as np

In [5]:
import pickle

In [6]:
import time

In [7]:
def train_model(train, valid, test, features,target):
    cv_folds = 5
    early_stopping_rounds = 50
    
    ### Forming copy of input datasets
    train_c, valid_c, test_c = train.copy(), valid.copy(), test.copy()
#     out_path = params['out_path']
#     predictors, target = list(params['predictors']), params['target']
    predictors, target = features, target
#     alg = XGBClassifier(
#                 learning_rate =params['learning_rate'],
#                 n_estimators=params['n_estimators'],
#                 max_depth=params['max_depth'],
#                 min_child_weight=params['min_child_weight'],
#                 gamma=params['gamma'],
#                 subsample=params['subsample'],
#                 colsample_bytree=params['colsample_bytree'],
#                 objective= 'binary:logistic',
#                 scale_pos_weight=params['scale_pos_weight'],
#                 seed=27)

    alg = XGBClassifier( colsample_bytree=0.8, gamma=0, learning_rate=0.1,
    max_depth=5, min_child_weight=1, n_estimators=5000, objective='binary:logistic',
    reg_alpha=0, reg_lambda=0, scale_pos_weight=(len(train) -train[df_target].sum())/train[df_target].sum(),
    seed=7, silent=True, subsample=0.8,n_jobs=7)
    
    eval_set = [(valid_c[predictors],valid_c[target])]
    alg.fit(train_c[predictors], train_c[target],eval_metric='auc', early_stopping_rounds=early_stopping_rounds, eval_set = eval_set, verbose = True)     
    
    valid_c.loc[:,'prob'] = alg.predict_proba(valid_c[predictors])[:,1]
    test_c.loc[:,'prob'] = alg.predict_proba(test_c[predictors])[:,1]
    
    valid_auc = roc_auc_score(np.array(valid_c.loc[:,target]), np.array(valid_c.loc[:,'prob']))
    test_auc = roc_auc_score(np.array(test_c.loc[:,target]), np.array(test_c.loc[:,'prob']))
    loss = 1 - valid_auc
    
#     if out_path != None:
#         f = open(os.path.join(out_path, "hyperopt_logs.txt"), "a+")
#         f.write("Valid AUC: {} and Test AUC: {}, loss: {}".format(valid_auc,test_auc,loss))
#         f.close()
    
    print ("Valid AUC: {} and Test AUC: {}, Loss: {}".format(valid_auc, test_auc, loss))
    return alg

In [8]:
def data(df,target):
    processed_df = pd.get_dummies(df,sparse=True)
    df_features = list(set(processed_df.columns) - set([target]))
#     train, valid, test = train_validate_test_split(processed_df)
    train = processed_df.ix[train_o.index]
    valid = processed_df.ix[valid_o.index]
    test = processed_df.ix[test_o.index]
    return train, valid, test, df_features, target

In [9]:
train_o = pd.read_csv('../data/train_insurance.csv',index_col = 0)
valid_o = pd.read_csv('../data/valid_insurance.csv',index_col = 0)
test_o = pd.read_csv('../data/test_insurance.csv',index_col = 0)

In [10]:
all_data = pd.concat([train_o,valid_o,test_o])

In [11]:
all_data.drop(['NewEstimateTotal','EstimateTotal'],axis=1,inplace=True)

In [12]:
train, valid, test, df_features, df_target = data(all_data,target='StartedFlag')

/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/ubuntu/insurance_izenda_analysis/insurace_izenda/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-

In [13]:
start_time = time.time()
model = train_model(train.head(300000), valid.head(60000), test, df_features, df_target)


with open('insurance_classify_300000_started_flag.pkl', 'wb') as f:
    pickle.dump(model, f)

f = open("started_flag_300000.txt", "w")
f.write("Took {} seconds".format(time.time()-start_time))
f.close() 

[0]	validation_0-auc:0.76198
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.77021
[2]	validation_0-auc:0.818435
[3]	validation_0-auc:0.82994
[4]	validation_0-auc:0.834366
[5]	validation_0-auc:0.83179
[6]	validation_0-auc:0.834991
[7]	validation_0-auc:0.836948
[8]	validation_0-auc:0.836438
[9]	validation_0-auc:0.835673
[10]	validation_0-auc:0.838376
[11]	validation_0-auc:0.838198
[12]	validation_0-auc:0.840488
[13]	validation_0-auc:0.840853
[14]	validation_0-auc:0.842525
[15]	validation_0-auc:0.84422
[16]	validation_0-auc:0.845286
[17]	validation_0-auc:0.846348
[18]	validation_0-auc:0.84696
[19]	validation_0-auc:0.8475
[20]	validation_0-auc:0.848143
[21]	validation_0-auc:0.84863
[22]	validation_0-auc:0.849548
[23]	validation_0-auc:0.850174
[24]	validation_0-auc:0.850732
[25]	validation_0-auc:0.851174
[26]	validation_0-auc:0.851718
[27]	validation_0-auc:0.852655
[28]	validation_0-auc:0.853225
[29]	validation_0-auc:0.853818
[30]	validation_0-auc:0.8

[266]	validation_0-auc:0.883561
[267]	validation_0-auc:0.883584
[268]	validation_0-auc:0.883619
[269]	validation_0-auc:0.883617
[270]	validation_0-auc:0.883645
[271]	validation_0-auc:0.883668
[272]	validation_0-auc:0.883764
[273]	validation_0-auc:0.883788
[274]	validation_0-auc:0.883788
[275]	validation_0-auc:0.883824
[276]	validation_0-auc:0.883847
[277]	validation_0-auc:0.883985
[278]	validation_0-auc:0.884008
[279]	validation_0-auc:0.884046
[280]	validation_0-auc:0.884082
[281]	validation_0-auc:0.884122
[282]	validation_0-auc:0.884153
[283]	validation_0-auc:0.884172
[284]	validation_0-auc:0.884235
[285]	validation_0-auc:0.884276
[286]	validation_0-auc:0.8843
[287]	validation_0-auc:0.884325
[288]	validation_0-auc:0.884358
[289]	validation_0-auc:0.884389
[290]	validation_0-auc:0.884423
[291]	validation_0-auc:0.884483
[292]	validation_0-auc:0.884516
[293]	validation_0-auc:0.884518
[294]	validation_0-auc:0.884556
[295]	validation_0-auc:0.884585
[296]	validation_0-auc:0.884607
[297]	vali

[524]	validation_0-auc:0.891122
[525]	validation_0-auc:0.891138
[526]	validation_0-auc:0.891188
[527]	validation_0-auc:0.891196
[528]	validation_0-auc:0.891201
[529]	validation_0-auc:0.891211
[530]	validation_0-auc:0.891225
[531]	validation_0-auc:0.891241
[532]	validation_0-auc:0.891267
[533]	validation_0-auc:0.891276
[534]	validation_0-auc:0.89129
[535]	validation_0-auc:0.891315
[536]	validation_0-auc:0.89133
[537]	validation_0-auc:0.891331
[538]	validation_0-auc:0.891362
[539]	validation_0-auc:0.891373
[540]	validation_0-auc:0.891376
[541]	validation_0-auc:0.891391
[542]	validation_0-auc:0.891399
[543]	validation_0-auc:0.891415
[544]	validation_0-auc:0.891431
[545]	validation_0-auc:0.891575
[546]	validation_0-auc:0.891592
[547]	validation_0-auc:0.891599
[548]	validation_0-auc:0.89161
[549]	validation_0-auc:0.891628
[550]	validation_0-auc:0.891628
[551]	validation_0-auc:0.891664
[552]	validation_0-auc:0.891672
[553]	validation_0-auc:0.891696
[554]	validation_0-auc:0.891706
[555]	valid

[782]	validation_0-auc:0.894676
[783]	validation_0-auc:0.894679
[784]	validation_0-auc:0.894696
[785]	validation_0-auc:0.894703
[786]	validation_0-auc:0.894725
[787]	validation_0-auc:0.894729
[788]	validation_0-auc:0.89474
[789]	validation_0-auc:0.894766
[790]	validation_0-auc:0.89478
[791]	validation_0-auc:0.894788
[792]	validation_0-auc:0.894797
[793]	validation_0-auc:0.894805
[794]	validation_0-auc:0.894809
[795]	validation_0-auc:0.894819
[796]	validation_0-auc:0.894815
[797]	validation_0-auc:0.894825
[798]	validation_0-auc:0.894829
[799]	validation_0-auc:0.894851
[800]	validation_0-auc:0.894854
[801]	validation_0-auc:0.894866
[802]	validation_0-auc:0.894875
[803]	validation_0-auc:0.894885
[804]	validation_0-auc:0.89492
[805]	validation_0-auc:0.89493
[806]	validation_0-auc:0.894936
[807]	validation_0-auc:0.894936
[808]	validation_0-auc:0.894941
[809]	validation_0-auc:0.894944
[810]	validation_0-auc:0.894949
[811]	validation_0-auc:0.89497
[812]	validation_0-auc:0.894968
[813]	validat

[1051]	validation_0-auc:0.897362
[1052]	validation_0-auc:0.897365
[1053]	validation_0-auc:0.897359
[1054]	validation_0-auc:0.897367
[1055]	validation_0-auc:0.897386
[1056]	validation_0-auc:0.897385
[1057]	validation_0-auc:0.897388
[1058]	validation_0-auc:0.897397
[1059]	validation_0-auc:0.897414
[1060]	validation_0-auc:0.897417
[1061]	validation_0-auc:0.897418
[1062]	validation_0-auc:0.89742
[1063]	validation_0-auc:0.897446
[1064]	validation_0-auc:0.897445
[1065]	validation_0-auc:0.897458
[1066]	validation_0-auc:0.897465
[1067]	validation_0-auc:0.897467
[1068]	validation_0-auc:0.897472
[1069]	validation_0-auc:0.897494
[1070]	validation_0-auc:0.8975
[1071]	validation_0-auc:0.897518
[1072]	validation_0-auc:0.897535
[1073]	validation_0-auc:0.897555
[1074]	validation_0-auc:0.897561
[1075]	validation_0-auc:0.897566
[1076]	validation_0-auc:0.897574
[1077]	validation_0-auc:0.897581
[1078]	validation_0-auc:0.897593
[1079]	validation_0-auc:0.897601
[1080]	validation_0-auc:0.897607
[1081]	valida

...

[1800]	validation_0-auc:0.90178
[1801]	validation_0-auc:0.901784
[1802]	validation_0-auc:0.901784
[1803]	validation_0-auc:0.901784
[1804]	validation_0-auc:0.901788
[1805]	validation_0-auc:0.901778
[1806]	validation_0-auc:0.901787
[1807]	validation_0-auc:0.901786
[1808]	validation_0-auc:0.90179
[1809]	validation_0-auc:0.901792
[1810]	validation_0-auc:0.901798
[1811]	validation_0-auc:0.901818
[1812]	validation_0-auc:0.901818
[1813]	validation_0-auc:0.901815
[1814]	validation_0-auc:0.901818
[1815]	validation_0-auc:0.901815
[1816]	validation_0-auc:0.901816
[1817]	validation_0-auc:0.90182
[1818]	validation_0-auc:0.90182
[1819]	validation_0-auc:0.901821
[1820]	validation_0-auc:0.901822
[1821]	validation_0-auc:0.901838
[1822]	validation_0-auc:0.901838
[1823]	validation_0-auc:0.901844
[1824]	validation_0-auc:0.901878
[1825]	validation_0-auc:0.901884
[1826]	validation_0-auc:0.901881
[1827]	validation_0-auc:0.901887
[1828]	validation_0-auc:0.901898
[1829]	validation_0-auc:0.901898
[1830]	validat

[2063]	validation_0-auc:0.902705
[2064]	validation_0-auc:0.902719
[2065]	validation_0-auc:0.90272
[2066]	validation_0-auc:0.90273
[2067]	validation_0-auc:0.902741
[2068]	validation_0-auc:0.902769
[2069]	validation_0-auc:0.902771
[2070]	validation_0-auc:0.902777
[2071]	validation_0-auc:0.902797
[2072]	validation_0-auc:0.902797
[2073]	validation_0-auc:0.902828
[2074]	validation_0-auc:0.90283
[2075]	validation_0-auc:0.902842
[2076]	validation_0-auc:0.902843
[2077]	validation_0-auc:0.902845
[2078]	validation_0-auc:0.902839
[2079]	validation_0-auc:0.902842
[2080]	validation_0-auc:0.90285
[2081]	validation_0-auc:0.902853
[2082]	validation_0-auc:0.902862
[2083]	validation_0-auc:0.902863
[2084]	validation_0-auc:0.902865
[2085]	validation_0-auc:0.902865
[2086]	validation_0-auc:0.902872
[2087]	validation_0-auc:0.90287
[2088]	validation_0-auc:0.902879
[2089]	validation_0-auc:0.902878
[2090]	validation_0-auc:0.902882
[2091]	validation_0-auc:0.902885
[2092]	validation_0-auc:0.902889
[2093]	validati

[2313]	validation_0-auc:0.903601
[2314]	validation_0-auc:0.903615
[2315]	validation_0-auc:0.903613
[2316]	validation_0-auc:0.903629
[2317]	validation_0-auc:0.903632
[2318]	validation_0-auc:0.903635
[2319]	validation_0-auc:0.903639
[2320]	validation_0-auc:0.903636
[2321]	validation_0-auc:0.903635
[2322]	validation_0-auc:0.903639
[2323]	validation_0-auc:0.903684
[2324]	validation_0-auc:0.903685
[2325]	validation_0-auc:0.903689
[2326]	validation_0-auc:0.90371
[2327]	validation_0-auc:0.903711
[2328]	validation_0-auc:0.903709
[2329]	validation_0-auc:0.903708
[2330]	validation_0-auc:0.90372
[2331]	validation_0-auc:0.903725
[2332]	validation_0-auc:0.90375
[2333]	validation_0-auc:0.90375
[2334]	validation_0-auc:0.903749
[2335]	validation_0-auc:0.903748
[2336]	validation_0-auc:0.903749
[2337]	validation_0-auc:0.903749
[2338]	validation_0-auc:0.903752
[2339]	validation_0-auc:0.903789
[2340]	validation_0-auc:0.903789
[2341]	validation_0-auc:0.903791
[2342]	validation_0-auc:0.903794
[2343]	validat

[2563]	validation_0-auc:0.904283
[2564]	validation_0-auc:0.904279
[2565]	validation_0-auc:0.904274
[2566]	validation_0-auc:0.90428
[2567]	validation_0-auc:0.904277
[2568]	validation_0-auc:0.904278
[2569]	validation_0-auc:0.904279
[2570]	validation_0-auc:0.904284
[2571]	validation_0-auc:0.904289
[2572]	validation_0-auc:0.904298
[2573]	validation_0-auc:0.904292
[2574]	validation_0-auc:0.90429
[2575]	validation_0-auc:0.904294
[2576]	validation_0-auc:0.904297
[2577]	validation_0-auc:0.904311
[2578]	validation_0-auc:0.904312
[2579]	validation_0-auc:0.904311
[2580]	validation_0-auc:0.904322
[2581]	validation_0-auc:0.904325
[2582]	validation_0-auc:0.904342
[2583]	validation_0-auc:0.904341
[2584]	validation_0-auc:0.904344
[2585]	validation_0-auc:0.904344
[2586]	validation_0-auc:0.904348
[2587]	validation_0-auc:0.904351
[2588]	validation_0-auc:0.904352
[2589]	validation_0-auc:0.904352
[2590]	validation_0-auc:0.904356
[2591]	validation_0-auc:0.904357
[2592]	validation_0-auc:0.904359
[2593]	valid

[2812]	validation_0-auc:0.904916
[2813]	validation_0-auc:0.90491
[2814]	validation_0-auc:0.904913
[2815]	validation_0-auc:0.904916
[2816]	validation_0-auc:0.904923
[2817]	validation_0-auc:0.904924
[2818]	validation_0-auc:0.904926
[2819]	validation_0-auc:0.904925
[2820]	validation_0-auc:0.904925
[2821]	validation_0-auc:0.90493
[2822]	validation_0-auc:0.90493
[2823]	validation_0-auc:0.904932
[2824]	validation_0-auc:0.90494
[2825]	validation_0-auc:0.904939
[2826]	validation_0-auc:0.904938
[2827]	validation_0-auc:0.904941
[2828]	validation_0-auc:0.904939
[2829]	validation_0-auc:0.904935
[2830]	validation_0-auc:0.904938
[2831]	validation_0-auc:0.90494
[2832]	validation_0-auc:0.904939
[2833]	validation_0-auc:0.904947
[2834]	validation_0-auc:0.904951
[2835]	validation_0-auc:0.904951
[2836]	validation_0-auc:0.904954
[2837]	validation_0-auc:0.904952
[2838]	validation_0-auc:0.904953
[2839]	validation_0-auc:0.90495
[2840]	validation_0-auc:0.90495
[2841]	validation_0-auc:0.904952
[2842]	validation

In [ ]:
# To load the pickle model\n",
with open('insurance_classify_100000_estimate_total.pkl', 'rb') as f:
    model = pickle.load(f)

In [20]:
valid.loc[:,'prob_up'] = model.predict_proba(valid[df_features])[:,1]

valid['prediction'] = np.where(valid.prob_up > 0.5,1,0)

accuracy_score(valid.StartedFlag, valid.prediction)

0.82179240007352

In [21]:
test.loc[:,'prob_up'] = model.predict_proba(test[df_features])[:,1]

test['prediction'] = np.where(test.prob_up > 0.5,1,0)

accuracy_score(test.StartedFlag, test.prediction)

0.8212032927282326

In [25]:
# Confusion Matrix on Test Data
metrics.confusion_matrix(test.StartedFlag, test.prediction)

array([[ 95387,  19929],
       [ 25792, 114607]])